In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from pypdf import PdfReader
import os


pdf_path = r"C:\Users\vince\OneDrive\Capstone\Final Data List.pdf"

reader = PdfReader(pdf_path)

text = "\n".join((p.extract_text() or "") for p in reader.pages)

names = re.findall(r"●\s*([A-Za-z\.\'\- ]+?)\s*(?:\(|\n)", text)

player_names = [n.strip() for n in names]


import pandas as pd
players_df = pd.DataFrame({"player_name": player_names})

players_df.head()



# now i have to add the position, tier?, and starred flag

positions = ["Quarterbacks","Running backs", "Wide Receivers", "Tight Ends",
             "Offensive Linemen","Defensive Linemen", "Linebackers", "Defensive Backs"]
positions = [" ".join(p.split()) for p in positions]

rows = []

current_position = None

for line in text.splitlines():
    line = " ".join(line.strip().split())
    if not line:
        continue

    for p in positions:
        if line.lower().startswith(p.lower()):
            current_position = p
            break
    if "●" in line:
        chunks = [c.strip() for c in line.split("●") if c.strip()]
        for c in chunks:
            starred = "*" in c
            c2 = c.replace("*", "").strip()
            m = re.search(r"\((\d)\)", c2)
            tier = int(m.group(1)) if m else None
    
            name = re.sub(r"\(\d\)", "", c2).strip()
            name = name.strip(" ?•")
    
            if name:
                rows.append({
                    "position": current_position,
                    "player_name": name,
                    "tier": tier,
                    "starred": starred
                })
            
players_table = pd.DataFrame(rows)


# adding a player_id column to the dataframe

players_table["players_name_clean"] = (players_table["player_name"].str.lower().str.replace(
    r"[^a-z\s]", "",regex = True).str.replace(r"\s+", " ", regex=True).str.strip()

)

players_table["player_id"] = range(1,len(players_table) + 1)

players_table

,position,player_name,tier,starred,players_name_clean,player_id
0,Quarterbacks,Patrick Mahomes,1.0,True,patrick mahomes,1
1,Quarterbacks,Drew Brees,1.0,True,drew brees,2
2,Quarterbacks,Cam Newton,2.0,True,cam newton,3
3,Quarterbacks,Johnny Manziel,3.0,True,johnny manziel,4
4,Quarterbacks,Zach Wilson,3.0,True,zach wilson,5
...,...,...,...,...,...,...
82,Defensive Backs,Malik Hooker,NaN,False,malik hooker,83
83,Defensive Backs,Jeff Okudah,NaN,False,jeff okudah,84
84,Defensive Backs,Marcus Peters,NaN,False,marcus peters,85
85,Defensive Backs,Budda Baker,NaN,False,budda baker,86
